<img src='logos/EUMETSAT_Logo_WhiteonBlue.png' alt='logo EUMETSAT' align='right' width='30%'></img>

## <span style="color:blue">**MTG LI data explore**</span>

<hr>

# LI Parallax and Light Travel Time Correction Demo

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
Notebook requirements:
    
Before using this notebook for the first time, be sure to:

* download and uncompress the file SSP_0_LUT.tar.gz from [**this link**](https://sftp.eumetsat.int/public/folder/UsCVknVOOkSyCdgpMimJNQ/User-Materials/MTGUP/LI_parallax_correction/)

* download LI data by following the instructions provided in the notebook [**1_6_MTG_LI_data_access.ipynb**](https://gitlab.eumetsat.int/eumetlab/data-services/eumdac_data_store/-/blob/master/1_6_MTG_LI_data_access.ipynb?ref_type=heads)

You may also want first to get familiar with LI data, for what you can use the notebook [**LI_FlashData_Explore.ipynb**](https://gitlab.eumetsat.int/eumetlab/weather/weather-labs/weather-labs-1/-/blob/master/Lab11_LI_data_display/LI_FlashData_Explore.ipynb?ref_type=heads)

</div>
<hr>

### About

This notebook demonstrates how to correct LI LFL or LGR locations for parallax and times for light travel time between cloud tops and the LI instrument. 

The parallax correction is needed as LI observes lightning optical emission at cloud top height. Depending on the location in the field of view, the season, and the storm type, this can be anywhere between ~6 km to ~20 km of altitude. This, in combination with the viewing angle of the MTG-I1 satellite, results in some offset between the lightning locations in the LI LFL/LGR product and the actual lightning/thunderstorm location on the ground. Generally, LI LFL/LGR locations appear further from the sub-satellite point (0&deg;N and 0&deg;E) than the actual lightning locations. The further from the sub-satellite point, the greater the offset. The offsets in Europe are expected to be up to tens of kilometers (increasing from southwest to northeast) with uncorrected lightning locations being too far north/northeast from actual storms.

The light travel time (LTT) correction is required as it takes some time for the photons to reach the LI instrument. The altitude of the geostationary orbit is approximately 36 000 km, resulting in ~120 ms photon travel time between the cloud tops and the LI instrument at the sub-satellite point. Due to Earth's curvature, the LTT increases up to ~140 ms along the edges of the visible disk. The LTT correction doesn't make a big difference in the operational use of the data, but becomes critical if LI groups/flashes are compared with the observations of other lightning location systems.   

As the LI L2 products are not corrected for the parallax and LTT effects, all users are provided with tools for introducting the necessary corrections. The tools consist of Montly Cloud top Height (CTH) look-up tables (LUTs) and a set of python functions. The monthly CTH LUTs are derived from SEVIRI 13-year ice-cloud top height climatology (thunderstorm tops are at the ice cloud alatitude). The python functions take the time and location of each LFL/LGR, derive its corresponding parallax and LTT corrections and apply these to the intial uncorrected LFL/LGR locations and times. The resulting corrected lightning locations and times are normally well in line with actual storm locations and the data of ground-based lightning location systems. There are a couple of situations where the parallax correction might not fully remove the initial location offset:

1) Very close to the edges of the LI field-of-view, especially along the edges of the visible Earth disk. The viewing angle of the satellite becomes parallel to the ground there and no meaningful parallax correction can be applied.

2) Untypical storms. An example would be a storm with unusually high or low cloud top height in the particular location and time of the year. Such large deviations from the CTH climatology will result in over- or undercorrection of the parallax effect. The issue has a possible workaround using real-time FCI cloud top height instead of the CTH climatology (this option will be provided later during the mission).   

All the examples below use the LFL product, but the code can be easily adjusted to read and correct the locations of LGRs. 

### Module Content:
- [1 - Load some LI LFL data](#LI_Load)
- [2 - Apply parallax and light travel time correction](#Parallax_correction)
- [3 - Display the effects of parallax correction](#Data_parallax)



<hr>

In [ ]:
from glob import glob
from satpy.scene import Scene
from AuxiliaryFunctions import plot_lmap
from ParallaxFunctions import apply_time_parallax_correction, parallax_shift_map

## <a id='LI_Load'></a>1. Load some LI LFL data

From the [EUMETSAT Data Store](https://data.eumetsat.int/data/map/EO:EUM:DAT:MSG:HRSEVIRI), we downloaded LI data sensed for a specific day. The **Data_folder** variable should point to the folder containing LI LFL products, unzipped by the LI data download Notebook. As the data is contained in the **BODY** files, and since the data files have 'BODY' in their filenames, we will use the wildcard *BODY* to search for the relevant files within the folder. Another wildcard `/*/` is needed as each product is unzipped into its own subdirectory.

We also add a specific constraint `2024091314` to only load files with the date of `2024-09-13` and the hour of `14` in their names. This will give us less data (about 1 hours and 10 minutes) and cleaner storm pattern to better see the impact of parallax correction. 

In [ ]:
Data_folder='/home/jupyter-cbarroso/weather-labs-1/Lab11_LI_data_display/LocalData/FLASHES/*BODY*2024091314*_N*.nc'
!ls /home/jupyter-cbarroso/weather-labs-1/Lab11_LI_data_display/LocalData/AF/*BODY*2024091314*_N*.nc

We save the list with the filenames in the folder into a variable called filenames, sort the list and print the filenames for checking:

In [ ]:
filenames = glob(Data_folder)
filenames.sort()
for fn in filenames:
    print(fn)

Next, we use the `Scene` constructor from the [satpy](https://satpy.readthedocs.io/en/stable/index.html) library to load the data files, using the `li_l2_nc` reader. Once loaded, a `Scene` object represents a single geographic region of data, typically at a single continuous time range.

In [ ]:
scn = Scene(reader="li_l2_nc", 
            filenames=filenames)

A `Scene` object loads all band information of a satellite image. With the function `available_dataset_names()`, you can see the available datasets available in the LI data file.

In [ ]:
scn.available_dataset_names()

For the parallax and time correction demo we will first need the raw uncorrected latitudes, longitudes and times of the flashes. Let's retrieve and save these as vars:

In [ ]:
latitude_dataset_name='latitude'
longitude_dataset_name='longitude'
time_dataset_name='flash_time'
scn.load([latitude_dataset_name, longitude_dataset_name, time_dataset_name])
latitude_dataset_uncorr = scn[latitude_dataset_name].compute()
longitude_dataset_uncorr = scn[longitude_dataset_name].compute()
time_dataset_uncorr = scn[time_dataset_name].compute()

We can also check the total number of flashes in the loaded data:

In [ ]:
print(f"Total of imported lightning observations: {len(latitude_dataset_uncorr)}")

## <a id='Parallax_correction'></a>2. Apply parallax and light travel time correction

For the parallax and time correction we need to first provide the directory where the LUT files are stored, and the date of the data we want to correct. 

In [ ]:
LUT_dir = '/data/Weather/LI_LUT_Parallax/SSP_0/'
date = '2024-09-13'

And now we can call the `apply_time_parallax_correction()` function, providing the LUT directory, date and the lat/lon and time arrays to correct as input arguments. As we want to demonstrate the impact of the correction below, we save the corrected arrays as new variabes `latitude_dataset_corr`, `longitude_dataset_corr` and `time_dataset_corr`. 

In [ ]:
latitude_dataset_corr, longitude_dataset_corr, time_dataset_corr = apply_time_parallax_correction(LUT_dir, date, latitude_dataset_uncorr, 
                                                                                                  longitude_dataset_uncorr, time_dataset_uncorr)

It can be seen that applying the correction takes some time. Therefore, for larger projects, involving longer periods of LI LGR/LFL data, it is recommended to apply the correction in smaller chunks (e.g. 1 hour at once) and then save the corrected data in e.g. daily LGR and LFL files. For example, in Python, one can store LGR/LFL data in `xarray.Dataset` format, saved in daily `netCDF` files. 

## <a id='Data_parallax'></a>3. Display the effects of parallax correction

The impact of parallax correction on the LGR/LFL locations is most obvious when a small area far from the sub-satellite point is displayed. Furthermore, keeping the time period relatively short (i.e. around 30-60 minutes) helps to reduce the impact of storm movemen, making the parallax correction effect easier to see. 

The maps below are set to display lightning data over Estonia during the period of 13:50 to 15:00 UTC on 2024-09-13. To zoom on a specific storm or focus on another Area Of Interest (AOI), you can do so by entering the latitude and longitude coordinates in the variable provided. The format should be **[lat_max, lat_min, lon_min, lon_max]**

In [ ]:
area_nswe = [60, 57, 21.5, 28.5]

First, let's plot the LI LFL locations before the parallax correction. 

In [ ]:
title = f'LI flashes before parallax correction'
plot_lmap(latitude_dataset_uncorr, longitude_dataset_uncorr, time_dataset_uncorr, area_nswe=area_nswe, title=title)

And now let's plot the LI LFL locations after the parallax correction. 

In [ ]:
title = f'LI flashes after parallax correction'
plot_lmap(latitude_dataset_corr, longitude_dataset_corr, time_dataset_corr, area_nswe=area_nswe, title=title)

There is also a very simple complementary function in the parallax correction module that allows one to visualize the parallax correction effect on one map. It plots the initial uncorrected lightning locations as red semi-transparent x-symbols and their corresponding parallax-corrected lightning locations as red non-transparent x-symbols. Red semi-transparent lines connect the initial uncorrected location and the final parallax corrected location of each lightning observation.  

Note that for clarity and readability, it is higly recommended to only plot a small amount of uncorrected-corrected location pairs at a time. The number of lightning location points that will be plotted is controlled by the `n_points` variable below. 

In [ ]:
title = 'LI parallax shift map'
n_points = 8
parallax_shift_map(latitude_dataset_uncorr, longitude_dataset_uncorr, latitude_dataset_corr, longitude_dataset_corr, 
                   area_nswe, title=title, maxcount=n_points)

<div style="border: 1px solid #ccc; padding: 10px; background-color: transparent;">

What is the direction of parallax correction in Estonia? Why?

Now change the `area_nswe` in section 3 above and investigate the direction and magnitude of parallax correction in the following areas:

1) Equatorial South America (4&deg;N - 1&deg;N, 66&deg;W - 62&deg;W)

2) The South Atlantic Ocean (33&deg;S - 37&deg;S, 43&deg;W - 38&deg;W)

3) The coast of central Africa near the sub-satellite point (6&deg;N - 3&deg;N, 4&deg;E - 9&deg;E)

4) The North Atlantic Ocean near the nortwest edge of LI field-of-view (55&deg;N - 52&deg;N, 26&deg;W - 22&deg;W)

5) The east coast of the Red Sea (17&deg;N - 15&deg;N, 42&deg;E - 45&deg;E)

6) The coast of Italy (44&deg;N - 42&deg;N, 9&deg;E - 12&deg;E)

Are the results expected, given the location of each of the areas, relative to the sub-satellite point at 0&deg;N and 0&deg;E?
